In [1]:
import logging
import pandas as pd
import numpy as np

from numpy import random
import gensim

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
df = pd.read_csv('severity_dataset.csv', delimiter=';', names=['ID', 'Guideword', 'Hazard', 'Detailed scenario', 'Hazardous event', 'Severity', 'Severity rationale'])
df = df[pd.notnull(df['Severity'])]
# shuffle the DataFrame rows
df = df.sample(frac=1).reset_index(drop=True)

# print(df.head(10))
# Calculate the total number of words in the 'Detailed scenario' column
df['Detailed scenario'].apply(lambda x: len(x.split(' '))).sum()

4157

In [3]:
def print_plot(index):
    example = df[df.index == index][['Detailed scenario', 'Severity']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Severity:', example[1])
print_plot(10)

"Ego vehicle is driving in a multi-lane one-way street with adjacent parking lanes and dotted line markings, a bicycle suddenly appears from the parking lane"
Severity: S2


In [8]:
# Download necessary NLTK data
nltk.download('stopwords')

REPLACE_BY_SPACE_RE = re.compile(r'[/(){}[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
    text: a string
    
    return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text  # HTML decoding
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwords from text
    return text

# Assuming df is already defined as shown in your previous code
df['Detailed scenario'] = df['Detailed scenario'].apply(clean_text)
print(df.head(10))


    ID   Guideword                                             Hazard  \
0  110    Omission  CAEM fails to produce a lateral motion request...   
1    4    Omission  CAEM fails to produce a lateral motion request...   
2   23    Omission  CAEM fails to produce a lateral motion request...   
3  143  Commission  CAEM produces a lateral motion request when th...   
4  153  Commission  CAEM produces a lateral motion request when th...   
5  193  Commission  CAEM produces a lateral motion request when th...   
6   97    Omission  CAEM fails to produce a lateral motion request...   
7   18  Commission  CAEM produces a lateral motion request when th...   
8   10    Omission  CAEM fails to produce a lateral motion request...   
9  151  Commission  CAEM produces a lateral motion request when th...   

                                   Detailed scenario  \
0  approaching tintersection yield signs truck st...   
1  sharp downhill curve low friction due wet road...   
2  sharp downhill curve low 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ranimkhojah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
